# Blood Glucose Control - EDA

**Name:** Kevin Lau



### Import Data

There are multiple data sets each corresponding to a person with diabetes. Please load the files into a single data set while retaining the ability to distinguish between the PWD. 

In [3]:
import pandas as pd

# Load CSV files
data_one= pd.read_csv('../data/gluroo_500030.csv')
data_two = pd.read_csv('../data/gluroo_679372.csv')

# Add Paitent ID Columns
data_one['patient_id'] = 500030
data_two['patient_id'] = 679372

# Combine
data = pd.concat([data_one, data_two])

# Assuming 'date' is a string and you want to extract parts of it
# Extract interval from the 27th to the 29th characters
df['interval'] = df['date'].apply(lambda x: x[27:29]) # Trimming head as per your logic

# Trim the tail and keep only the first 19 characters of the date
df['date'] = df['date'].apply(lambda x: x[:19]) 

# Convert the 'date' column to a proper datetime object
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S')

# Display the first few rows to check the results
print(df.head())

                 date  sender_id    bgl  bgl_date_millis text template  \
0 2024-05-01 00:00:27        NaN  114.0              NaN  NaN      NaN   
1 2024-05-01 00:05:27        NaN  117.0              NaN  NaN      NaN   
2 2024-05-01 00:10:26        NaN  113.0              NaN  NaN      NaN   
3 2024-05-01 00:15:27        NaN  112.0              NaN  NaN      NaN   
4 2024-05-01 00:20:27        NaN  113.0              NaN  NaN      NaN   

  msg_type affects_fob affects_iob  dose_units  food_g  food_glycemic_index  \
0      NaN         NaN         NaN         NaN     NaN                  NaN   
1      NaN         NaN         NaN         NaN     NaN                  NaN   
2      NaN         NaN         NaN         NaN     NaN                  NaN   
3      NaN         NaN         NaN         NaN     NaN                  NaN   
4      NaN         NaN         NaN         NaN     NaN                  NaN   

  dose_automatic  fp_bgl  message_basal_change __typename trend  patient_id  \
0

C:\Users\User\AppData\Local\Temp\ipykernel_42252\3452621658.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### Two Data Visualizations

Create at least two visualizations that are of interest to you. They can be things that would be useful information for a PWD to know, or they could be useful for informing future modeling work. If you aren't sure what to visualize try recreating common diabetes data visualizations that you can find on the internet. 

### Ask a question(s) about the data and/or diabetes!
When creating the visualizations you probably had some questions about managing T1D. Ask some questions that might interest you and our diabetes experts will do their best to answer them for you! *Consider this an AMAA about diabetes.* 

1. [Question 1]
2. [Question 2]

### Final Task

Thanks for showing interest in our project! We plan to incorporate the best/most interesting visualizations into a substack post in the future. 

Please remember to create a pull request with your updated template, don't forget to rename the file to something that we can identify you with, e.g: `eda_[your_name].ipynb`